# Import Libraries

In [ ]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
import hashlib
from pinecone import Pinecone
from langchain.llms import OpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
import json
import ast
from rapidfuzz import fuzz
from datetime import datetime
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.autonotebook import tqdm

c:\Codes\Python\notebooks\echo_chatbot\.chatbot\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


## Initialization

In [3]:
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

# Pinecone Initialization
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index("scs")
namespaces = ["Kickoff Meeting", "Project Meeting"]

# OpenAI Initialization
client = OpenAI(api_key=OPENAI_API_KEY)
EMBEDDINGS = OpenAIEmbeddings(model='text-embedding-3-small', openai_api_key=OPENAI_API_KEY)
LLM = ChatOpenAI(temperature=0, model_name="gpt-4-turbo", openai_api_key=OPENAI_API_KEY)

C:\Users\Zeck Lowe\AppData\Local\Temp\ipykernel_24780\662142765.py:10: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  client = OpenAI(api_key=OPENAI_API_KEY)


# Query

In [47]:
query = "Who are present in kickoff meeting and what did they contribute?"

# Generate Embeddings

In [40]:
def get_embeddings(text):
    """
    This function returns a list of the embeddings for a given query
    """
    text_embeddings = EMBEDDINGS.embed_query(text)
    print("Generating Embeddings: Done!")
    return text_embeddings

query_embeddings = get_embeddings(text=query)
print(query_embeddings)

Generating Embeddings: Done!
[-0.032539449632167816, 0.040285613387823105, 0.05508384481072426, -0.011771950870752335, -0.03995244577527046, 0.002146506914868951, 0.0392305813729763, 0.016366899013519287, 0.002141301054507494, -0.013541908003389835, 0.036121007055044174, -0.05133569985628128, -0.011771950870752335, -0.029735280200839043, -0.020947962999343872, -0.021961351856589317, -0.04761531949043274, 0.005396634340286255, 0.006611310876905918, 0.012209233827888966, 0.03217851743102074, 0.047643084079027176, -0.013180974870920181, 0.009536946192383766, -0.02470999024808407, 0.03387212008237839, -0.05105805769562721, -0.008232036605477333, 0.05078041926026344, -0.0002533468068577349, 0.012813101522624493, -0.03145664930343628, -0.0007019094773568213, 0.012056532315909863, -0.024501759558916092, -0.0006134116556495428, -0.009828467853367329, 0.01979575678706169, -0.017116528004407883, 0.004081313032656908, 0.03276155889034271, -0.04647699370980263, -0.0019296003738418221, -0.019476469

# Get Namespace

### Semantic Similarity

In [41]:
def get_most_similar_namespace(query, namespaces, threshold=0.05):
    """
    Rank namespaces by semantic similarity to the query.
    """
    # Get query embeddings
    query_embeddings = get_embeddings(query)
    print(query_embeddings)

    # Get embeddings for each namespace in list
    namespace_embeddings = {ns: get_embeddings(ns) for ns in namespaces}
    print(namespace_embeddings)

    # Compute similarities
    similarities = {
        ns: cosine_similarity([query_embeddings], [embedding])[0][0] for ns, embedding in namespace_embeddings.items()
    }
    print(similarities.items)

    # Rank namespaes by similarity score
    ranked_namespaces = sorted(similarities.items(), key=lambda x: x[1], reverse=True)
    print(ranked_namespaces)

    # Check if the top two are close in similarity
    top_two = ranked_namespaces[:2]
    print(top_two)
    if len(top_two) > 1 and abs(top_two[0][1] - top_two[1][1]) < threshold:
        return None, top_two # Ambiguous case, return for user clarification
    
    return ranked_namespaces[0][0], ranked_namespaces # Returns most similar namespace and the ranked_namespaces with its embeddings

namespace, ranked = get_most_similar_namespace(query, namespaces)

Generating Embeddings: Done!
[-0.032539449632167816, 0.040285613387823105, 0.05508384481072426, -0.011771950870752335, -0.03995244577527046, 0.002146506914868951, 0.0392305813729763, 0.016366899013519287, 0.002141301054507494, -0.013541908003389835, 0.036121007055044174, -0.05133569985628128, -0.011771950870752335, -0.029735280200839043, -0.020947962999343872, -0.021961351856589317, -0.04761531949043274, 0.005396634340286255, 0.006611310876905918, 0.012209233827888966, 0.03217851743102074, 0.047643084079027176, -0.013180974870920181, 0.009536946192383766, -0.02470999024808407, 0.03387212008237839, -0.05105805769562721, -0.008232036605477333, 0.05078041926026344, -0.0002533468068577349, 0.012813101522624493, -0.03145664930343628, -0.0007019094773568213, 0.012056532315909863, -0.024501759558916092, -0.0006134116556495428, -0.009828467853367329, 0.01979575678706169, -0.017116528004407883, 0.004081313032656908, 0.03276155889034271, -0.04647699370980263, -0.0019296003738418221, -0.019476469

### User Clarification Loop

In [42]:
def clarify_with_user(ambiguous_namespaces):
    """
    Ask the user to clarify when multiple namespaces are similar.
    """
    options = [ns[0] for ns in ambiguous_namespaces]
    print(options)
    print(f"Did you mean:\n1. {options[0]}\n2. {options[1]}")

    # Simulate user input for demonstration
    user_choice = int(input("Please choose 1 or 2: "))-1
    return options[user_choice]

### Integration

In [43]:
def resolve_namespace(query, namespaces):
    """
    Resolves the namespace by either selecting the most similar one or prompting the user for clarification.
    """
    namespace, ranked = get_most_similar_namespace(query, namespaces)
    print(namespaces, ranked)

    if namespace:
        print(f"Selected namespace: {namespace}")
        return namespace
    else:
        print("Ambiguity detected!")
        return clarify_with_user(ranked)
    
resolved_namespace = resolve_namespace(query, namespaces)
print(f"Namespace: {resolved_namespace}")

Generating Embeddings: Done!
[-0.03253113478422165, 0.04030308127403259, 0.05509753152728081, -0.011768943630158901, -0.039886724203825, 0.002140754135325551, 0.03924831748008728, 0.016334960237145424, 0.002126875566318631, -0.013538448140025139, 0.036139536648988724, -0.051405858248472214, -0.011775882914662361, -0.02972768433392048, -0.02092873491346836, -0.021969620138406754, -0.04765867069363594, 0.005447299685329199, 0.0066304393112659454, 0.012171419337391853, 0.03214253857731819, 0.04760315641760826, -0.013184547424316406, 0.009513691999018192, -0.024717556312680244, 0.03383571282029152, -0.05101725831627846, -0.008236872963607311, 0.0507674477994442, -0.0002558843116275966, 0.012802889570593834, -0.0314486138522625, -0.0006947909714654088, 0.01203263457864523, -0.024495501071214676, -0.0006375422817654908, -0.00981901865452528, 0.01981845684349537, -0.017126033082604408, 0.004076800774782896, 0.03275319188833237, -0.04652063548564911, -0.0019325771136209369, -0.0194853730499744

# Query Pinecone Index

In [44]:
def query_pinecone_index(query_embeddings, meeting_title, top_k=5, include_metadata=True):
    """
    Query a Pinecone index.
    """
    # Build filter conditions directly for Pinecone
    filter_conditions = {}

    # Include date and meeting title if specified
    if meeting_title.lower() != 'unknown':
        filter_conditions['title'] = meeting_title

    # Query Pinecone using the build filter conditions
    query_response = index.query(
        vector=query_embeddings,
        filter=filter_conditions,
        top_k=top_k,
        include_metadata=include_metadata,
        namespace=meeting_title) # Filter based on metadata

    print("Querying Pinecone Index: Done!")
    return query_response

answers = query_pinecone_index(query_embeddings=query_embeddings, meeting_title=resolved_namespace)
print(answers)

Querying Pinecone Index: Done!
{'matches': [], 'namespace': 'Kickoff Meeting', 'usage': {'read_units': 1}}


# Combining Text from Multiple Document Matches

In [39]:
text_answer = " ".join([doc['metadata']['text'] for doc in answers['matches']])
print(text_answer)

[00:00:00] John: Good morning, everyone. Thank you for joining today's kickoff meeting for our new     software development project. We'll be discussing the project scope, timelines, and
responsibilities. Let's get started with a quick round of introductions. I'll go first. I'm John, the     project manager. I'll be overseeing the project and ensuring we stay on track. Alice, would you     like to go next?
[00:00:20] research, collaborate with developers. Sara develop and execute the testing plan, ensure
software quality. John oversee the project, coordinate between teams, ensure timely delivery.
If there are no further questions, we'll conclude the meeting. Thank you all for your time and let's
make this project a success.
[00:03:50]
All: Thank you, John. overall architecture and development of the software. Looking forward to working with all of you.
[00:00:35]
Bob: Hi, I'm Bob, the UI/UX designer. I'll be handling the design aspects of the software, making
sure it's user-friendly an

# Helper Functions

In [35]:
def output_parser(output):
    """
    Helps parses the LLM output, prints it, and returns it.
    """
    print("\n" + output.content + "\n")

    return output.content

## Prompt

In [ ]:
prompt = f"""You are a meeting facilitator.
        This user will ask you a questions about the conversation of the meeting.
        Use following piece of context to answer the question.
        If you don't know the answer, just say you don't know.
        Keep the answer complete and concise.
        Context: {text_answer}
        Question: {query}"""

print(prompt)


        You are a meeting facilitator.
        This user will ask you a questions about the conversation of the meeting.
        Use following piece of context to answer the question.
        If you don't know the answer, just say you don't know.
        Keep the answer complete and concise.
        Context: {context}
        Question: {question} 

input_variables=['context', 'question'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="\n        You are a meeting facilitator.\n        This user will ask you a questions about the conversation of the meeting.\n        Use following piece of context to answer the question.\n        If you don't know the answer, just say you don't know.\n        Keep the answer complete and concise.\n        Context: {context}\n        Question: {question}"), additional_kwargs={})]


# LLM

In [ ]:
def better_query_response(prompt):
    """
    This function returns a better response using LLM
    """
    better_answer = LLM.invoke(prompt)
    print("Generating Better Response: Done!")
    return better_answer

final_answer = better_query_response(prompt=prompt)
print(final_answer.content)

Generating Better Response: Done!
{context}
{question}


# Chatbot Response

In [46]:
def Chatbot(query, meeting_title=None, date=None):
    print(query)
    meeting_title = resolve_namespace(query, namespaces)

    query_embeddings = get_embeddings(text=query)

    answers = query_pinecone_index(
        query_embeddings=query_embeddings,
        meeting_title=meeting_title,
        )
    print(answers)

    text_answers = " ".join([doc['metadata']['text'] for doc in answers['matches']])
    print(text_answers)
    final_answer = better_query_response(prompt=prompt)
    return final_answer.content

response = Chatbot(query)
print(response)

Who are present in kickoff meeting?
Generating Embeddings: Done!
[-0.032539449632167816, 0.040285613387823105, 0.05508384481072426, -0.011771950870752335, -0.03995244577527046, 0.002146506914868951, 0.0392305813729763, 0.016366899013519287, 0.002141301054507494, -0.013541908003389835, 0.036121007055044174, -0.05133569985628128, -0.011771950870752335, -0.029735280200839043, -0.020947962999343872, -0.021961351856589317, -0.04761531949043274, 0.005396634340286255, 0.006611310876905918, 0.012209233827888966, 0.03217851743102074, 0.047643084079027176, -0.013180974870920181, 0.009536946192383766, -0.02470999024808407, 0.03387212008237839, -0.05105805769562721, -0.008232036605477333, 0.05078041926026344, -0.0002533468068577349, 0.012813101522624493, -0.03145664930343628, -0.0007019094773568213, 0.012056532315909863, -0.024501759558916092, -0.0006134116556495428, -0.009828467853367329, 0.01979575678706169, -0.017116528004407883, 0.004081313032656908, 0.03276155889034271, -0.04647699370980263, 

## RAG Chain

In [49]:
class chat_templates:
    def final_rag_template():
        prompt = """
            You are a meeting facilitator.
            This user will ask you a questions about the conversation of the meeting.
            Use following piece of context to answer the question.
            If you don't know the answer, just say you don't know.
            Keep the answer complete and concise.
            Context: {context}
            Here are some background questions and answers that will help you answer the question: {qa_pairs}
            Question: {question}
        """
        
        return prompt
    
    def decomposition_template():
        prompt = """
            Break the following user question into smaller, more specific questions.
            Provide these subquestions separated by newlines. 
            Do not rephrase if you see unknown terms.
            Question: {question}
            subquestions:
        """

        return prompt
    
    def qa_template():
        prompt = """
            Answer the question in the following context:\n{context}\n\nQuestion: {subquestion}
        """

        return prompt
        

prompt_template = ChatPromptTemplate.from_template(chat_templates.final_rag_template())
decomposition_template = ChatPromptTemplate.from_template(chat_templates.decomposition_template())
qa_template = ChatPromptTemplate.from_template(chat_templates.qa_template())

def decompose_question(question):
    """
    Decomposes a complex question into smaller questions.
    """
    prompt = decomposition_template.format(question=question)
    response = LLM.invoke(prompt)
    subquestions = response.content.split("\n")

    return subquestions

query_embeddings = get_embeddings(text=query)
answers = query_pinecone_index(
        query_embeddings=query_embeddings,
        meeting_title=namespace,
        )
text_answers = " ".join([doc['metadata']['text'] for doc in answers['matches']])

def generate_qa_pairs(subquestions, context):
    """Generates QA pairs by answering each subquestion."""
    qa_pairs = []
    for subquestion in subquestions:
        context = context
        rag_prompt = qa_template.format(context=context, subquestion=subquestion)
        answer = LLM.invoke(rag_prompt)
        qa_pairs.append((subquestion, answer))
    return qa_pairs

def build_final_answer(question, context, qa_pairs):
    """Builds a final answer by integrating the context and QA pairs."""
    qa_pairs_str = "\n".join([f"Q: {q}\nA: {a}" for q, a in qa_pairs])
    final_prompt = prompt_template.format(context=context, qa_pairs=qa_pairs_str, question=question)
    final_response = LLM.invoke(final_prompt)
    return final_response

def decomposition_query_process(question, text_answers):
    """Implements decomposition query"""
    subquestions = decompose_question(question)

    qa_pairs = generate_qa_pairs(subquestions, text_answers)
    print(qa_pairs)

    final_answer = build_final_answer(question, text_answers, qa_pairs)

    return output_parser(final_answer)

result = decomposition_query_process(question=query, text_answers=text_answers)

Generating Embeddings: Done!
Querying Pinecone Index: Done!
[('Who are the participants in a kickoff meeting?', AIMessage(content='The participants in the kickoff meeting are John, Alice, Bob, and Sara.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 518, 'total_tokens': 534, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-turbo-2024-04-09', 'system_fingerprint': 'fp_cfb5f9efa7', 'finish_reason': 'stop', 'logprobs': None}, id='run-cea6b1e0-c372-4ce0-b90c-f55523508017-0', usage_metadata={'input_tokens': 518, 'output_tokens': 16, 'total_tokens': 534})), ('', AIMessage(content='What are the main responsibilities of each team member in this software development project?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'com